-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# Optimizing Data Storage with Delta Lake

Databricks supports a number of optimizations for clustering data and improving directory and file skipping while scanning and loading data files. While some of these optimizations will use the word "index" in describing the process used, these indices differ from the algorithms many users will be familiar with from traditional SQL database systems.

In this notebook we'll explore how optional data storage and optimization settings on Delta Lake interact with file size and data skipping.

## Learning Objectives
By the end of this lessons, students will be able to:
- Describe default behavior for statistics collection and file skipping on Delta Lake
- Identify columns well-suited to partitioning
- Use **`OPTIMIZE`** to compact small files
- Apply Z-order to optimize file skipping on high cardinality fields
- Use Bloom filters to speed up queries on fields with arbitrary text

In [0]:
%run ../Includes/Classroom-Setup-1.2

Dropping the database "dbacademy_odl_user_771624_databrickslabs_com_adewd_1_2"
Removing the working directory "dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.2"

Creating the database "dbacademy_odl_user_771624_databrickslabs_com_adewd_1_2"
Creating raw_data...(17 seconds / 6,443,331 records)

Predefined Paths:
 DA.paths.working_dir: dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.2
 DA.paths.user_db: dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.2/1_2.db
 DA.paths.checkpoints: dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.2/_checkpoints

Predefined tables in dbacademy_odl_user_771624_databrickslabs_com_adewd_1_2:
 raw_data

Validating datasets...(1 seconds)

Setup completed in 25 seconds

## Create a Delta Table

The following CTAS statement creates a simple, unpartitioned external Delta Lake table from a sample dataset

In [0]:
%sql
CREATE OR REPLACE TABLE no_part_table
LOCATION "${da.paths.working_dir}/no_part_table"
AS SELECT * FROM raw_data

num_affected_rows,num_inserted_rows


## Schema Considerations
When configuring tables in Delta Lake, make sure you consider the following.

### Precision
Both numeric and datetime types should be stored with the correct precision specified to:
1. Ensure integrity with source systems
1. Maintain precision and avoid rounding errors for downstream queries
1. Avoid unnecessary storage costs (note the significant differences in bytes for <a href="https://spark.apache.org/docs/latest/sql-ref-datatypes.html" target="_blank">numeric types</a>)

### Datetime Filtering
If data will be frequently filtered by year, year & month, day of week, date, or another datetime value, consider calculating these values at write time if not present in original data. (Pushdown filters work best on fields present in a table).

### Case Sensitivity
Spark does not differentiate case by default.

### Un-Nest Important Fields for Filtering
Extract fields that might be useful for indexing or filtering to increase performance.

### Place Important Fields Early in the Schema
Fields that will be used for filtering and optimizations should appear at the beginning of the schema declaration.

## File Skipping with Delta Lake File Statistics

By default, Delta Lake will capture statistics on the first 32 columns that appear in a table. These statistics indicate:
- the total number of records per file
- minimum value in each column 
- maximum value in each column
- null value counts for each of the columns

**NOTE**: These statistics are generally uninformative for string fields with very high cardinality (such as free text fields). You can omit these fields from statistic collection by <a href="https://docs.databricks.com/delta/optimizations/file-mgmt.html#data-skipping" target="_blank">moving them outside the first 32 columns or changing the number of columns on which statistics are collected</a>.  Nested fields count when determining the first 32 columns, for example 4 struct fields with 8 nested fields will total to the 32 columns.

## Reviewing Statistics in the Transaction Log

Statistics are recorded in the Delta Lake transaction log files. Files are initially committed in the JSON format, but are compacted to Parquet format automatically to accelerate metadata retrieval.

Transaction logs can be viewed in the **`_delta_log`** directory within the table location.

In [0]:
files = dbutils.fs.ls(f"{DA.paths.working_dir}/no_part_table/_delta_log")
display(files)

path,name,size,modificationTime
dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.2/no_part_table/_delta_log/00000000000000000000.crc,00000000000000000000.crc,2204,1666766124000
dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.2/no_part_table/_delta_log/00000000000000000000.json,00000000000000000000.json,3185,1666766122000
dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.2/no_part_table/_delta_log/__tmp_path_dir/,__tmp_path_dir/,0,1666766124000


JSON log files can be easily parsed with Spark. Statistics for each file are accessible in the **`add`** column.

When a query with a selective filter (**`WHERE`** clause) is executed against a Delta Lake table, the query optimizer uses the information stored in the transaction logs to identify files that **may** contain records matching the conditional filter.

In [0]:
display(spark.read.json(f"{DA.paths.working_dir}/no_part_table/_delta_log/00000000000000000000.json"))

add,commitInfo,metaData,protocol
null,"List(1024-143331-6vol2yy0, Databricks-Runtime/10.4.x-scala2.12, false, WriteSerializable, List(4382774100893083), CREATE OR REPLACE TABLE AS SELECT, List(3, 238238810, 6443331), List(null, false, [], {}), 1666766121913, e78224de-9c85-438b-beef-aedf6f6ff85e, 8453215174696142, odl_user_771624@databrickslabs.com)",null,null
null,null,null,"List(1, 2)"
null,null,"List(1666766109072, List(parquet), 6eaac8be-c416-4e14-9550-e59f2417483c, List(), {""type"":""struct"",""fields"":[{""name"":""key"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""value"",""type"":""binary"",""nullable"":true,""metadata"":{}},{""name"":""topic"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""partition"",""type"":""long"",""nullable"":true,""metadata"":{}},{""name"":""offset"",""type"":""long"",""nullable"":true,""metadata"":{}},{""name"":""timestamp"",""type"":""long"",""nullable"":true,""metadata"":{}}]})",null
"List(true, 1666766120000, part-00000-72004204-2eb6-4422-bb4b-18e7288c8c2e-c000.snappy.parquet, 130757670, {""numRecords"":3538050,""minValues"":{""key"":""101589"",""topic"":""bpm"",""partition"":0,""offset"":65663680,""timestamp"":1575849600082},""maxValues"":{""key"":""198867"",""topic"":""bpm"",""partition"":4,""offset"":67529701,""timestamp"":1576454399607},""nullCount"":{""key"":0,""value"":0,""topic"":0,""partition"":0,""offset"":0,""timestamp"":0}}, List(1666766112000000, 268435456))",null,null,null
"List(true, 1666766116000, part-00001-b4e5d5ce-1c20-44a2-b919-390f65649734-c000.snappy.parquet, 82372714, {""numRecords"":2228629,""minValues"":{""key"":""101589"",""topic"":""bpm"",""partition"":0,""offset"":65663686,""timestamp"":1575849600130},""maxValues"":{""key"":""198867"",""topic"":""bpm"",""partition"":4,""offset"":67529703,""timestamp"":1576454399814},""nullCount"":{""key"":0,""value"":0,""topic"":0,""partition"":0,""offset"":0,""timestamp"":0}}, List(1666766112000001, 268435456))",null,null,null
"List(true, 1666766112000, part-00002-46b3e19f-d637-4001-9fb5-5fdb25796303-c000.snappy.parquet, 25108426, {""numRecords"":676652,""minValues"":{""key"":""101589"",""topic"":""bpm"",""partition"":0,""offset"":12824,""timestamp"":1575853785037},""maxValues"":{""key"":""49835"",""topic"":""workout"",""partition"":4,""offset"":67669938,""timestamp"":1576515593355},""nullCount"":{""key"":0,""value"":0,""topic"":0,""partition"":0,""offset"":0,""timestamp"":0}}, List(1666766112000002, 268435456))",null,null,null


Note that columns used for Z-ordering need to have statistics collected. Even without additional optimization metrics, statistics will always be leveraged for file skipping.

**NOTE**: Calculating statistics on free-form text fields (product reviews, user messages, etc.) can be time consuming. For best performance, set these fields later in the schema and <a href="https://docs.databricks.com/delta/optimizations/file-mgmt.html#data-skipping" target="_blank">change the number of columns that statistics are collected on</a>.

## Partitioning Delta Lake Tables

The partitioning method used in Delta Lake is similar to that used by Hive or Spark with Parquet (recall that Delta Lake data files are stored as Parquet).

When a column is used to partition a table, each unique value found in that column will create a separate directory for data. When choosing partition columns, it's good to consider the following:
1. How will the table be used?
   - **Partitioning can help optimize performance for operational OR analytic queries (rarely both)**
1. How many total values will be present in a column?
   - **Low cardinality fields should be used for partitioning**
1. How many total records will share a given value for a column?
   - **Partitions should be at least 1 GB in size (or larger depending on total table size)**
1. Will records with a given value continue to arrive indefinitely?
   - **Discrete datetime values can allow partitions to be optimized and archived once late-arriving data is processed**

**NOTE**: When in doubt, do not partition data at all. Other data skipping features in Delta Lake can achieve similar speeds as partitioning, but data that is over-partitioned or incorrectly partitioned will suffer greatly (and require a full rewrite of all data files to remedy).

Columns representing measures of time and low-cardinality fields used frequently in queries are good candidates for partitioning. The code below creates a table partitioned by date using <a href="https://docs.databricks.com/delta/delta-batch.html#deltausegeneratedcolumns" target="_blank">generated columns</a>. Generated columns will be stored the same way other columns are, but will be calculated at write time using the logic provided when the table was defined.

In [0]:
%sql
CREATE OR REPLACE TABLE date_part_table (
  key STRING,
  value BINARY,
  topic STRING,
  partition LONG,
  offset LONG,
  timestamp LONG,
  p_date DATE GENERATED ALWAYS AS (CAST(CAST(timestamp/1000 AS timestamp) AS DATE))
)
PARTITIONED BY (p_date)
LOCATION '${da.paths.working_dir}/date_part_table'

In [0]:
(spark.table("raw_data")
      .write.mode("append")
      .saveAsTable("date_part_table"))

In [0]:
%sql
select * from raw_data limit 10;

key,value,topic,partition,offset,timestamp
177966,eyJkZXZpY2VfaWQiOiAxNzc5NjYsICJ0aW1lIjogMTU3NTg0OTYwMCwgImhlYXJ0cmF0ZSI6IDQwLjY1NzI4MTg0Njg4MjQ2fQ==,bpm,1,65777162,1575849606429
183816,eyJkZXZpY2VfaWQiOiAxODM4MTYsICJ0aW1lIjogMTU3NTg0OTU5MSwgImhlYXJ0cmF0ZSI6IDQ5LjIxMjcwODU2NzYxNjE3fQ==,bpm,1,65777158,1575849604367
183816,eyJkZXZpY2VfaWQiOiAxODM4MTYsICJ0aW1lIjogMTU3NTg0OTU5MSwgImhlYXJ0cmF0ZSI6IDQ5LjIxMjcwODU2NzYxNjE3fQ==,bpm,1,65777159,1575849604367
196248,eyJkZXZpY2VfaWQiOiAxOTYyNDgsICJ0aW1lIjogMTU3NTg0OTU5MywgImhlYXJ0cmF0ZSI6IDQ3LjAzNTU3NzY1Njg0NDk3fQ==,bpm,3,65734875,1575849603460
107952,eyJkZXZpY2VfaWQiOiAxMDc5NTIsICJ0aW1lIjogMTU3NTg0OTYwMCwgImhlYXJ0cmF0ZSI6IDY2LjcyNTU2NDU3MDUxOTk4fQ==,bpm,2,66117479,1575849609506
187047,eyJkZXZpY2VfaWQiOiAxODcwNDcsICJ0aW1lIjogMTU3NTg0OTYwMiwgImhlYXJ0cmF0ZSI6IDQ3LjQ5NDQ0NjExODc5ODE0NX0=,bpm,2,66117499,1575849612839
191498,eyJkZXZpY2VfaWQiOiAxOTE0OTgsICJ0aW1lIjogMTU3NTg0OTU5OSwgImhlYXJ0cmF0ZSI6IDU1LjM4MTcyMjU5NTE5MDgzfQ==,bpm,3,65734887,1575849610659
197930,eyJkZXZpY2VfaWQiOiAxOTc5MzAsICJ0aW1lIjogMTU3NTg0OTYxNCwgImhlYXJ0cmF0ZSI6IDU2Ljc3OTM5ODMzNjkzNzI3fQ==,bpm,0,66168162,1575849626891
124989,eyJkZXZpY2VfaWQiOiAxMjQ5ODksICJ0aW1lIjogMTU3NTg0OTYxOCwgImhlYXJ0cmF0ZSI6IDQ1Ljc2MDQyMjAyODg5NDUyfQ==,bpm,4,65663719,1575849628126
182045,eyJkZXZpY2VfaWQiOiAxODIwNDUsICJ0aW1lIjogMTU3NTg0OTYyNCwgImhlYXJ0cmF0ZSI6IDU1Ljk0MTIxMzg0MDM1MTAwNH0=,bpm,0,66168170,1575849632843


Listing the location used for the table reveals that the unique values in the partition column are used to generate data directories. Note that the Parquet format used to store the data for Delta Lake leverages these partitions directly when determining column value (the column values for **`p_date`** are not stored redundantly within the data files).

In [0]:
files = dbutils.fs.ls(f"{DA.paths.working_dir}/date_part_table")
display(files)

path,name,size,modificationTime
dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.2/date_part_table/_delta_log/,_delta_log/,0,1666766147000
dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.2/date_part_table/p_date=2019-12-09/,p_date=2019-12-09/,0,1666766137000
dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.2/date_part_table/p_date=2019-12-10/,p_date=2019-12-10/,0,1666766138000
dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.2/date_part_table/p_date=2019-12-11/,p_date=2019-12-11/,0,1666766140000
dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.2/date_part_table/p_date=2019-12-12/,p_date=2019-12-12/,0,1666766141000
dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.2/date_part_table/p_date=2019-12-13/,p_date=2019-12-13/,0,1666766142000
dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.2/date_part_table/p_date=2019-12-14/,p_date=2019-12-14/,0,1666766143000
dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.2/date_part_table/p_date=2019-12-15/,p_date=2019-12-15/,0,1666766144000
dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.2/date_part_table/p_date=2019-12-16/,p_date=2019-12-16/,0,1666766133000


The data in this table look largely the same, except that more files were written because of the separation of data into separate directories based on the date.

In [0]:
path = f"{DA.paths.working_dir}/date_part_table/_delta_log/00000000000000000001.json"
df = spark.read.json(path)
display(df)

add,commitInfo
null,"List(1024-143331-6vol2yy0, Databricks-Runtime/10.4.x-scala2.12, true, WriteSerializable, List(4382774100893083), WRITE, List(22, 238301228, 6443331), List(Append, []), 0, 1666766145626, 8f6e1933-5342-4b9e-a547-1e382570edb6, 8453215174696142, odl_user_771624@databrickslabs.com)"
"List(true, 1666766138000, List(2019-12-09), p_date=2019-12-09/part-00000-a2f9a91f-743b-47ec-9a13-a4b3fb16feb8.c000.snappy.parquet, 16754984, {""numRecords"":454713,""minValues"":{""key"":""101589"",""topic"":""bpm"",""partition"":0,""offset"":65663680,""timestamp"":1575849600082},""maxValues"":{""key"":""198867"",""topic"":""bpm"",""partition"":4,""offset"":66337436,""timestamp"":1575935999953},""nullCount"":{""key"":0,""value"":0,""topic"":0,""partition"":0,""offset"":0,""timestamp"":0}}, List(1666766132000000, 268435456))",null
"List(true, 1666766140000, List(2019-12-10), p_date=2019-12-10/part-00000-6a133646-a78c-4b28-9f69-d3df89f2933e.c000.snappy.parquet, 17755755, {""numRecords"":481329,""minValues"":{""key"":""101589"",""topic"":""bpm"",""partition"":0,""offset"":65811801,""timestamp"":1575936000155},""maxValues"":{""key"":""198867"",""topic"":""bpm"",""partition"":4,""offset"":66517643,""timestamp"":1576022399353},""nullCount"":{""key"":0,""value"":0,""topic"":0,""partition"":0,""offset"":0,""timestamp"":0}}, List(1666766132000001, 268435456))",null
"List(true, 1666766141000, List(2019-12-11), p_date=2019-12-11/part-00000-f9023686-dc77-4947-b4ae-eaaa7ccee2e6.c000.snappy.parquet, 18294067, {""numRecords"":495652,""minValues"":{""key"":""101589"",""topic"":""bpm"",""partition"":0,""offset"":65963028,""timestamp"":1576022400208},""maxValues"":{""key"":""198867"",""topic"":""bpm"",""partition"":4,""offset"":66715864,""timestamp"":1576108799938},""nullCount"":{""key"":0,""value"":0,""topic"":0,""partition"":0,""offset"":0,""timestamp"":0}}, List(1666766132000002, 268435456))",null
"List(true, 1666766142000, List(2019-12-12), p_date=2019-12-12/part-00000-6f29e6af-720a-4b19-aeb0-1bfdf79a02fe.c000.snappy.parquet, 18714037, {""numRecords"":506357,""minValues"":{""key"":""101589"",""topic"":""bpm"",""partition"":0,""offset"":66093548,""timestamp"":1576108800055},""maxValues"":{""key"":""198867"",""topic"":""bpm"",""partition"":4,""offset"":66923343,""timestamp"":1576195199809},""nullCount"":{""key"":0,""value"":0,""topic"":0,""partition"":0,""offset"":0,""timestamp"":0}}, List(1666766132000003, 268435456))",null
"List(true, 1666766143000, List(2019-12-13), p_date=2019-12-13/part-00000-b80c0bf0-0c1d-4767-a9b6-d49115e41960.c000.snappy.parquet, 19154072, {""numRecords"":518067,""minValues"":{""key"":""101589"",""topic"":""bpm"",""partition"":0,""offset"":66233752,""timestamp"":1576195200400},""maxValues"":{""key"":""198867"",""topic"":""bpm"",""partition"":4,""offset"":67131060,""timestamp"":1576281599780},""nullCount"":{""key"":0,""value"":0,""topic"":0,""partition"":0,""offset"":0,""timestamp"":0}}, List(1666766132000004, 268435456))",null
"List(true, 1666766144000, List(2019-12-14), p_date=2019-12-14/part-00000-13c9009d-356c-4906-83eb-8245e1bfbbf0.c000.snappy.parquet, 19482182, {""numRecords"":526032,""minValues"":{""key"":""101589"",""topic"":""bpm"",""partition"":0,""offset"":66392265,""timestamp"":1576281600062},""maxValues"":{""key"":""198867"",""topic"":""bpm"",""partition"":4,""offset"":67332404,""timestamp"":1576367999936},""nullCount"":{""key"":0,""value"":0,""topic"":0,""partition"":0,""offset"":0,""timestamp"":0}}, List(1666766132000005, 268435456))",null
"List(true, 1666766145000, List(2019-12-15), p_date=2019-12-15/part-00000-73016fa4-3aa0-432a-b8ad-afc321e5e518.c000.snappy.parquet, 20625155, {""numRecords"":555900,""minValues"":{""key"":""101589"",""topic"":""bpm"",""partition"":0,""offset"":66571381,""timestamp"":1576368000166},""maxValues"":{""key"":""198867"",""topic"":""bpm"",""partition"":4,""offset"":67529701,""timestamp"":1576454399607},""nullCount"":{""key"":0,""value"":0,""topic"":0,""partition"":0,""offset"":0,"

When running a query that filters data on a column used for partitioning, partitions not matching a conditional statement will be skipped entirely. Delta Lake also have several operations (including **`OPTIMIZE`** commands) that can be applied at the partition level.

Note that because data files will be separated into different directories based on partition values, files cannot be combined or compacted across these partition boundaries. Depending on the size of data in a given table, the "right size" for a partition will vary, but if most partitions in a table will not contain at least 1GB of data, the table is likely over-partitioned, which will lead to slowdowns for most general queries.

In [0]:
%sql
SELECT p_date, COUNT(*) 
FROM date_part_table 
GROUP BY p_date

p_date,count(1)
2019-12-09,710179
2019-12-12,844004
2019-12-14,875254
2019-12-10,751787
2019-12-16,675831
2019-12-11,796317
2019-12-13,863766
2019-12-15,926193


## Computing Stats

Users can <a href="https://docs.databricks.com/spark/latest/spark-sql/language-manual/sql-ref-syntax-aux-analyze-table.html" target="_blank">manually specify relational entities for which statistics should be calculated with **`ANALYZE`**</a>. While analyzing a table or a subset of columns for a table is not equivalent to indexing, it can allow the query optimizer to select more efficient plans for operations such as joins.

Statistics can be collected for all tables in a database, a specific table, a partition of a table, or a subset of columns in a table.

Below, statistics are computed for the **`timestamp`** column.

In [0]:
%sql
ANALYZE TABLE no_part_table 
COMPUTE STATISTICS FOR COLUMNS timestamp

These statistics can be seen by running **`DESCRIBE EXTENDED`** on the table and column.

In [0]:
%sql
DESCRIBE EXTENDED no_part_table timestamp

info_name,info_value
col_name,timestamp
data_type,bigint
comment,NULL
min,1575849600082
max,1576515593355
num_nulls,0
distinct_count,5513668
avg_col_len,8
max_col_len,8
histogram,NULL


## File Compaction
Delta Lake supports the **`OPTIMIZE`** operation, which performs file compaction. The <a href="https://docs.databricks.com/delta/optimizations/file-mgmt.html#autotune-based-on-table-size" target="_blank">target file size can be auto-tuned</a> by Databricks, and is typically between 256 MB and 1 GB depending on overall table size.

Note that data files cannot be combined across partitions. As such, some tables will benefit from not using partitions to minimize storage costs and total number of files to scan.

**NOTE**: Optimization schedules will vary depending on the nature of the data and how it will be used downstream. Optimization can be scheduled for off-hours to reduce competition for resources with important workloads. Delta Live Tables has added functionality to automatically optimize tables.

## Z-Ordering

Z-ordering is a technique to collocate related information in the same set of files. This co-locality is automatically used by Databricks data-skipping algorithms to dramatically reduce the amount of data that needs to be read.

Don't worry about <a href="https://en.wikipedia.org/wiki/Z-order_curve" target="_blank">the math</a> (tl;dr: Z-order maps multidimensional data to one dimension while preserving locality of the data points).

Multiple columns can be used for Z-ordering, but the algorithm loses some efficiency with each additional column. The best columns for Z-ordering are high cardinality columns that will be used commonly in queries.

Z-ordering must be executed at the same time as **`OPTIMIZE`**, as it requires rewriting data files.

Below is the code to Z-order and optimize the **`date_part_table`** by **`timestamp`** (this might be useful for regular queries within granular time ranges).

In [0]:
%sql
OPTIMIZE date_part_table
ZORDER BY (timestamp)

path,metrics
dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.2/date_part_table,"List(7, 21, List(26167898, 34361978, 3.045588042857143E7, 7, 213191163), List(6184, 20625155, 1.015359242857143E7, 21, 213225441), 8, List(minCubeSize(107374182400), List(0, 0), List(22, 238301228), 0, List(21, 213225441), 7, null), 1, 22, 1, false)"


Note that the metrics will provide an overview of what happened during the operation; reviewing the table history will also provide this information.

In [0]:
%sql
DESCRIBE HISTORY date_part_table

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
2,2022-10-26T06:36:17.000+0000,8453215174696142,odl_user_771624@databrickslabs.com,OPTIMIZE,"Map(predicate -> [], zOrderBy -> [""timestamp""], batchId -> 0, auto -> false)",null,List(4382774100893083),1024-143331-6vol2yy0,1,SnapshotIsolation,false,"Map(numRemovedFiles -> 21, numRemovedBytes -> 213225441, p25FileSize -> 27728944, minFileSize -> 26167898, numAddedFiles -> 7, maxFileSize -> 34361978, p75FileSize -> 32416191, p50FileSize -> 31190787, numAddedBytes -> 213191163)",null,Databricks-Runtime/10.4.x-scala2.12
1,2022-10-26T06:35:45.000+0000,8453215174696142,odl_user_771624@databrickslabs.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(4382774100893083),1024-143331-6vol2yy0,0,WriteSerializable,true,"Map(numFiles -> 22, numOutputRows -> 6443331, numOutputBytes -> 238301228)",null,Databricks-Runtime/10.4.x-scala2.12
0,2022-10-26T06:35:28.000+0000,8453215174696142,odl_user_771624@databrickslabs.com,CREATE OR REPLACE TABLE,"Map(isManaged -> false, description -> null, partitionBy -> [""p_date""], properties -> {})",null,List(4382774100893083),1024-143331-6vol2yy0,null,WriteSerializable,true,Map(),null,Databricks-Runtime/10.4.x-scala2.12


## Bloom Filter Indexes

While Z-order provides useful data clustering for high cardinality data, it's often most effective when working with queries that filter against continuous numeric variables.

Bloom filters provide an efficient algorithm for probabilistically identifying files that may contain data using fields containing arbitrary text. Appropriate fields would include hashed values, alphanumeric codes, or free-form text fields.

Bloom filters calculate indexes that indicate the likelihood a given value **could** be in a file; the size of the calculated index will vary based on the number of unique values present in the field being indexed and the configured tolerance for false positives.

**NOTE**: A false positive would be a file that the index thinks could have a matching record but does not. Files containing data matching a selective filter will never be skipped; false positives just mean that extra time was spent scanning files without matching records.

Looking at the distribution for the **`key`** field, this is an ideal candidate for this technique.

In [0]:
%sql
SELECT key, count(*) FROM no_part_table GROUP BY key ORDER BY count(*) ASC

key,count(1)
19548,1
42643,1
36590,1
28776,1
17217,1
21896,1
19888,1
43810,1
49661,1
32018,2


The code below sets a Bloom filter index on the **`key`** field with a false positivity allowance of 0.1%.

In [0]:
%sql
CREATE BLOOMFILTER INDEX
ON TABLE date_part_table
FOR COLUMNS(key OPTIONS (fpp=0.1, numItems=200))

## Hands-On

Go through the process of Z-ordering and adding a Bloom filter index to the **`no_part_table`**. Review the history for the table to confirm the operations were successful.

In [0]:
%sql 
OPTIMIZE no_part_table
ZORDER BY (timestamp);



path,metrics
dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.2/no_part_table,"List(1, 3, List(238237233, 238237233, 2.38237233E8, 1, 238237233), List(25108426, 130757670, 7.941293666666667E7, 3, 238238810), 0, List(minCubeSize(107374182400), List(0, 0), List(3, 238238810), 0, List(3, 238238810), 1, null), 1, 3, 0, false)"


In [0]:
%sql
CREATE BLOOMFILTER INDEX
ON TABLE no_part_table
FOR COLUMNS(key OPTIONS (fpp=0.1, numItems=200))

In [0]:
%sql
DESCRIBE HISTORY no_part_table

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
2,2022-10-26T06:54:26.000+0000,8453215174696142,odl_user_771624@databrickslabs.com,CREATE BLOOMFILTER INDEX,"Map(columns -> [{""name"":""key"",""type"":""string"",""nullable"":true,""metadata"":{""delta.bloomFilter.fpp"":0.1,""delta.bloomFilter.numItems"":200,""delta.bloomFilter.maxExpectedFpp"":1.0,""delta.bloomFilter.enabled"":true}}])",null,List(4382774100893083),1024-143331-6vol2yy0,1,WriteSerializable,true,Map(),null,Databricks-Runtime/10.4.x-scala2.12
1,2022-10-26T06:53:43.000+0000,8453215174696142,odl_user_771624@databrickslabs.com,OPTIMIZE,"Map(predicate -> [], zOrderBy -> [""timestamp""], batchId -> 0, auto -> false)",null,List(4382774100893083),1024-143331-6vol2yy0,0,SnapshotIsolation,false,"Map(numRemovedFiles -> 3, numRemovedBytes -> 238238810, p25FileSize -> 238237233, minFileSize -> 238237233, numAddedFiles -> 1, maxFileSize -> 238237233, p75FileSize -> 238237233, p50FileSize -> 238237233, numAddedBytes -> 238237233)",null,Databricks-Runtime/10.4.x-scala2.12
0,2022-10-26T06:35:22.000+0000,8453215174696142,odl_user_771624@databrickslabs.com,CREATE OR REPLACE TABLE AS SELECT,"Map(isManaged -> false, description -> null, partitionBy -> [], properties -> {})",null,List(4382774100893083),1024-143331-6vol2yy0,null,WriteSerializable,false,"Map(numFiles -> 3, numOutputRows -> 6443331, numOutputBytes -> 238238810)",null,Databricks-Runtime/10.4.x-scala2.12


Note: Adding a bloom filter will not create the filter for existing file-parts.  Only newly written files will have a filter created.  Optimizing an unoptimized delta table typically will result in writing all new files and therefore populate the filter.  But if the table is already optimized this will not work and you may need to copy the table instead.

Run the following cell to delete the tables and files associated with this lesson.

In [0]:
DA.cleanup()

Dropping the database "dbacademy_odl_user_771624_databrickslabs_com_adewd_1_2"
Removing the working directory "dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/1.2"
Validating datasets...(1 seconds)

-sandbox
&copy; 2022 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>